##### 0.1. Importing all the needed libraries

In [1]:
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split as tts
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torch import optim
import numpy as np
import pandas as pd
import torch
import optuna
import os

##### 0.2. Setting some global settins

In [2]:
%matplotlib ipympl

##### 0.3. Importing the data file


##### 1.1. Building the Neural Network

In [3]:
def clp_network(trial=False, n_layers=-1, units=[]):
    
    # checking if the network is custom or automatic
    custom_network = not trial and len(units) == n_layers

    # setting automatic number of layers
    if not custom_network: n_layers = trial.suggest_int("n_layers", 2, 6)
    
    layers = []
    in_features = 6
    
    # for each layer of the network
    for i in range(n_layers):
        
        # setting the number of units in the layer
        if custom_network: out_features = units[i]
        else: out_features = trial.suggest_int("n_units_l{}".format(i), 4, 10)
        
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.LeakyReLU())

        in_features = out_features
        
    layers.append(nn.Linear(in_features, 2))
    layers.append(nn.LeakyReLU())
    
    return nn.Sequential(*layers)

##### 1.2. Making a Dataset class

In [4]:
class clp_dataset(Dataset):

    def __init__(self, df):
        self.labels = [label for label in df['loyalty']]
        self.features = df.drop(columns=['loyalty'], axis=1).values.tolist()

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        return np.array(self.labels[idx])

    def get_batch_features(self, idx):
        return np.array(self.features[idx])

    def __getitem__(self, idx):
        batch_features = self.get_batch_features(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_features, batch_y

##### 1.3. Training and evaluation of the model

In [5]:
def train_and_evaluate(param, model, trial=False):

    # extracting parameters
    if 'n_epochs' in param: n_epochs = param['n_epochs']
    else: n_epochs = 50 # DEFAULT VALUE

    if 'batch_size' in param: batch_size = param['batch_size']
    else: batch_size = 32 # DEFAULT VALUE
    
    # importing data from file
    data_path = os.path.join(os.getcwd(), 'Data\\processed_customer_data.csv')
    data = pd.read_csv(data_path)
    data.drop(columns='ID', inplace=True)
    
    # separating train and dev sets
    train_data, val_data = tts(data, test_size = 0.2)
    train, val = clp_dataset(train_data), clp_dataset(val_data)

    # creating data loaders
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=batch_size)

    # activating CUDA
    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    # setting up the optimizer and evaluator objects
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(optim, param['optimizer'])(model.parameters(), lr= param['learning_rate'])

    # moving process to CUDA GPU
    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    # epochs
    for epoch_num in range(n_epochs):

            # accuracy and loss of the epoch
            total_acc_train = 0
            total_loss_train = 0

            # one training step
            for train_input, train_label in train_dataloader:

                train_label = train_label.to(device)
                train_input = train_input.to(device)

                output = model(train_input.float())
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            # repeating everything for the dev set
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    val_input = val_input.to(device)

                    output = model(val_input.float())

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            # calculating the accuract of the model
            accuracy = total_acc_val/len(val_data)
            
            # adding pruning mechanism
            if trial:
                trial.report(accuracy, epoch_num)

                if trial.should_prune():
                    raise optuna.exceptions.TrialPruned()

    return accuracy, model

##### 1.4. Making the objective function

In [6]:
# initializing models variable
models = dict()

# defining objective function of optuna study
def objective(trial, n_epochs=False, batch_size=False):
     
     if not n_epochs: n_epochs = trial.suggest_int("n_epochs", 30, 180)
     if not batch_size: batch_size = 2**trial.suggest_int("batch_size", 1, 6)
     
     # setting trial parameters
     params = {
              'learning_rate': trial.suggest_loguniform('learning_rate', 1e-6, 1e-1),
              'optimizer': trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"]),
              'n_epochs': n_epochs,
              'batch_size': batch_size
              }
     # creating the model
     model = clp_network(trial)

     # training the model
     accuracy, model = train_and_evaluate(params, model, trial)

     # saving the model (if it has a higher accuract than the previous ones)
     if accuracy > models['accuracy']:
          models['accuracy'] = accuracy
          models['model'] = model
     
     return accuracy

In [7]:
# initializing some variables
models = dict()
models['accuracy'] = 0

# initializing and calling optimize for each study
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(), pruner=optuna.pruners.HyperbandPruner())
study.optimize(lambda trial: objective(trial), n_trials=1000, show_progress_bar=True)

# extracting the best trial of the study
best_params = study.best_trial
param_dict = best_params.params
param_dict['batch_size'] = 2**param_dict['batch_size']
# saving details of the trial
params_dict = dict()
best_model_index = best_params.number
params_dict['model'] = models['model']

params_dict['params'] = param_dict
params_dict['accuracy'] = best_params.value

print(f"Study report:\n\t", end='\n\n')
print(f"\tn_epoch = {param_dict['n_epochs']}, batch_size = {param_dict['batch_size']}")
print(f"\tAccuracy = {params_dict['accuracy']}")

[I 2022-08-31 15:34:07,829] A new study created in memory with name: no-name-c560c107-eaa9-454d-96cd-cb9d002c210a
c:\Users\Mohsen\anaconda3\envs\CLP\lib\site-packages\optuna\progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/1000 [00:00<?, ?it/s]

[I 2022-08-31 15:34:40,645] Trial 0 finished with value: 0.7071428571428572 and parameters: {'n_epochs': 114, 'batch_size': 3, 'learning_rate': 0.0005431859416185268, 'optimizer': 'RMSprop', 'n_layers': 2, 'n_units_l0': 7, 'n_units_l1': 6}. Best is trial 0 with value: 0.7071428571428572.
[I 2022-08-31 15:37:53,578] Trial 1 finished with value: 0.7357142857142858 and parameters: {'n_epochs': 116, 'batch_size': 1, 'learning_rate': 0.007308251427623791, 'optimizer': 'RMSprop', 'n_layers': 6, 'n_units_l0': 4, 'n_units_l1': 6, 'n_units_l2': 9, 'n_units_l3': 5, 'n_units_l4': 7, 'n_units_l5': 8}. Best is trial 1 with value: 0.7357142857142858.
[I 2022-08-31 15:37:59,558] Trial 2 finished with value: 0.7142857142857143 and parameters: {'n_epochs': 125, 'batch_size': 6, 'learning_rate': 1.1106687209243363e-05, 'optimizer': 'SGD', 'n_layers': 5, 'n_units_l0': 4, 'n_units_l1': 5, 'n_units_l2': 10, 'n_units_l3': 6, 'n_units_l4': 5}. Best is trial 1 with value: 0.7357142857142858.
[I 2022-08-31 15:

In [8]:
torch.save({
    'model_state_dict': params_dict['model'].state_dict(),
    'accuracy': params_dict['accuracy'],
    'architecture': params_dict['params'] 
    }, 'Models\\customer_loyalty_prediction_auto.pt')

print(f"Final model:\n\t"
    + f"Accuracy: {params_dict['accuracy']}\n\t"
    + f"Model Saved Path: 'Models\\customer_loyalty_prediction_auto.pt'")

Final model:
	Accuracy: 0.8785714285714286
	Model Saved Path: 'Models\customer_loyalty_prediction_auto.pt'
